In [1]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

import os, argparse, sys
# sys.path.append('./')
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)
# importing scripts in scripts folder
from scripts import config as src

**** scripts/config.py IMPORTED!!!
**** PROJECT FOLDER ,  /home/pgundogdu/projects/signalization_prior_knowledge_based_nn


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import LeavePGroupsOut, train_test_split
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score

epochs_default=100 # for model design
batch_default=10   # for model design
val_split=0.1      # for model design
rand_state = 91    # dataset split
test_size = 0.3    # train_test_split
kf_split = 5       # KFold

from numba import cuda
from tensorflow import keras



import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model

from sklearn.cluster import KMeans

In [3]:
loc_output = 'models/NN/exper_mouse/leavePgroupsOut'
model_path = src.define_folder(loc_output)
model_path
save_path = src.define_folder('models/NN/exper_mouse/leavePgroupsOut/clustering')

FOLDER information,  models/NN/exper_mouse/leavePgroupsOut/
FOLDER information,  models/NN/exper_mouse/leavePgroupsOut/clustering/


In [4]:
df_learning = pd.read_csv('data/external/exper_mouse/1-3_integrated_NNtraining.txt'
                          , sep='\t'
                          , index_col=0).T.drop(columns='Weight')#.reset_index()
df_learning = df_learning.rename(columns={'Label':'cell_type'})
df_learning = df_learning[sorted(df_learning.columns[1:]) + [df_learning.columns[0] ]]
print(df_learning.shape)
df_learning.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
S1,0,0,0.0,103.853,0.0586557,0,3.97772,136.672,0,28.4994,...,0,0,0.278639,0,0.0,0.0,297.919,37.6855,0,BMDC
S2,0,0,0.0,0,0,0,0.00742256,440.725,0,3.15802,...,0,147.538,0,0,0.0,0.0,0.242888,0,0,BMDC
S3,0,0,0.0,0,0,0,0.0261848,118.676,0,0,...,0,17.7033,0.0863634,0,0.0,0.0,6.75353,0.132011,0.0176152,BMDC
S4,0,0,0.0,0,0,0,0,355.623,0,9.10864,...,0,0.537516,0.0636703,0,0.0,0.0,0.274689,9.72489,0,BMDC
S5,0,0,0.0,0,0,0,0,151.626,0,19.3458,...,0,128.928,0,0.204522,0.0,0.0,59.2561,44.4307,0,BMDC


In [7]:
df_learning.iloc[:, :-1].astype('float32').sum(axis=1)

S1            534007.18750
S2            613230.87500
S3            518435.15625
S4            610496.68750
S5            566544.12500
                  ...     
GSM1112733    739720.75000
GSM1112692    756580.12500
GSM1112726    734681.87500
GSM1112723    727859.31250
GSM1112529    704337.75000
Length: 402, dtype: float32

In [5]:
scaler = MinMaxScaler()
df_learning.iloc[:, :-1] = scaler.fit_transform(df_learning.iloc[:, :-1])
print(df_learning.shape)
df_learning.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
S1,0,0,0,0.193008,0.000101553,0,0.0968245,0.10984,0,0.102434,...,0,0,0.00143474,0,0,0,0.413223,0.652245,0,BMDC
S2,0,0,0,0,0,0,0.000180678,0.354199,0,0.0113507,...,0,0.302217,0,0,0,0,0.000336894,0,0,BMDC
S3,0,0,0,0,0,0,0.000637382,0.0953768,0,0,...,0,0.0362633,0.000444694,0,0,0,0.00936737,0.00228479,9.9952e-05,BMDC
S4,0,0,0,0,0,0,0,0.285804,0,0.0327387,...,0,0.00110105,0.000327845,0,0,0,0.000381003,0.168314,0,BMDC
S5,0,0,0,0,0,0,0,0.121858,0,0.0695334,...,0,0.264095,0,0.000588339,0,0,0.0821901,0.768988,0,BMDC


In [6]:
df_scaled = df_learning.reset_index(drop=True)
print(df_scaled.shape)
df_scaled.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
0,0,0,0,0.193008,0.000101553,0,0.0968245,0.10984,0,0.102434,...,0,0,0.00143474,0,0,0,0.413223,0.652245,0,BMDC
1,0,0,0,0,0,0,0.000180678,0.354199,0,0.0113507,...,0,0.302217,0,0,0,0,0.000336894,0,0,BMDC
2,0,0,0,0,0,0,0.000637382,0.0953768,0,0,...,0,0.0362633,0.000444694,0,0,0,0.00936737,0.00228479,9.9952e-05,BMDC
3,0,0,0,0,0,0,0,0.285804,0,0.0327387,...,0,0.00110105,0.000327845,0,0,0,0.000381003,0.168314,0,BMDC
4,0,0,0,0,0,0,0,0.121858,0,0.0695334,...,0,0.264095,0,0.000588339,0,0,0.0821901,0.768988,0,BMDC


In [7]:
n_groups_ = [2,4,6,8]
ohe = OneHotEncoder()
# n_groups_ = [4]

X = df_scaled.iloc[:, :-1].values
y = df_scaled.iloc[:, -1:].values
y_ohe = ohe.fit_transform(y).toarray()
groups = df_scaled.iloc[:, -1].values


In [8]:
df_pathway_signaling = pd.read_csv('data/processed/pbk_layer_mmu.txt', index_col=0)
df_pathway_metsig = pd.read_csv('data/processed/pbk_layer_mmu_metsig.txt', index_col=0)
df_ppi = pd.read_csv('data/processed/pbk_layer_ppi.txt', index_col=0)
df_ppitf = pd.read_csv('data/processed/pbk_layer_ppitf.txt', index_col=0)

print(df_pathway_signaling.shape)
print(df_pathway_metsig.shape)
print(df_ppi.shape)
print(df_ppitf.shape)

df_scaled = pd.concat([df_scaled.iloc[:, :-1].astype(np.float32).reset_index(drop=True), df_scaled.iloc[:, -1:].reset_index(drop=True)], axis=1)
print(df_scaled.info())
df_scaled_with_pathway_signaling = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_pathway_signaling.index) )+ ['cell_type'] )]
df_scaled_with_pathway_metsig = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_pathway_metsig.index) )+ ['cell_type'] )]
df_scaled_with_ppi = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_ppi.index) )+ ['cell_type'] )]
df_scaled_with_ppitf = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_ppitf.index) )+ ['cell_type'] )]

print(df_scaled_with_pathway_signaling.shape)
print(df_scaled_with_pathway_metsig.shape)
print(df_scaled_with_ppi.shape)
print(df_scaled_with_ppitf.shape)

(3737, 92)
(3922, 250)
(3553, 348)
(9437, 696)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Columns: 9438 entries, a1bg to cell_type
dtypes: float32(9437), object(1)
memory usage: 14.5+ MB
None
(402, 1646)
(402, 3923)
(402, 3554)
(402, 9438)


In [9]:
def def_clustering_experiment(X_train, X_test, model_path_, cluster_):
    
    K.clear_session()
    model_loc = os.path.join(model_path_)
    model_load = keras.models.load_model(model_loc)
    model_= Model(inputs=model_load.layers[0].input # first layer
                  , outputs=model_load.layers[-2].output) # the last layer before output
#         print(model_.summary())
    y_pred_all, array_y_test = [], []

    nn_last_layer_training = model_.predict(X_train)
    nn_last_layer_testing = model_.predict(X_test)
    kmeans = KMeans(n_clusters=cluster_).fit(nn_last_layer_training)
    kmeans_predict = kmeans.predict(nn_last_layer_testing)

    K.clear_session()
    y_pred_all.append(kmeans_predict)
#         print(len(y_pred_all))

    return(list(y_pred_all[0]))

In [10]:
# FULLY CONNECTED 1-LAYER-100ppi
# df_scaler_use = df_scaled.copy()
# design = 'full_gene_space_1_layer_dense100+ppi'

# FULLY CONNECTED 1-LAYER-100ppitf
df_scaler_use = df_scaled.copy()
design = 'full_gene_space_1_layer_dense100+ppitf_ss'

In [11]:
seed(1)
df_result = pd.DataFrame()
for i_pgroups in n_groups_[1:2]:
    
    print(i_pgroups)
    splitter = LeavePGroupsOut(n_groups=i_pgroups)
    splits = list(splitter.split(X, y, groups))
    ids = np.random.choice(len(splits), 20).tolist()
    list_random_selected = [splits[i] for i in ids]
    
    design_name = design+'-LeavePGroupOut_'+str(i_pgroups)

    df_all_per_cell_out = pd.DataFrame()
    for i in range(len(list_random_selected)):
        print('EXPERIMENT -- ', str(i+1), '/', str(len(list_random_selected)))
        ohe_kept = OneHotEncoder()
        X_train, y_train, X_test, y_test = [], [], [], []

        df_kept_groups = df_scaler_use[df_scaler_use.index.isin(list_random_selected[i][0])]
        df_leave_groups = df_scaler_use[df_scaler_use.index.isin(list_random_selected[i][1])]


        X_kept = df_kept_groups.iloc[:, :-1].values
        y_kept = df_kept_groups.iloc[:, -1].values#y_ohe[list_random_selected[i][0]]

        
        X_leave = df_leave_groups.iloc[:, :-1].values
        y_leave = df_leave_groups.iloc[:, -1:].values#y_ohe[list_random_selected[i][0]]

        
        X_train.append(X_kept)
        X_test.append(X_leave)
        y_test.append(y_leave)
        
#         y_test.append([item for sublist in ytest for item in sublist])

        
        model_path_info = os.path.join(model_path, 'cell_out_'+str(i_pgroups) ,design+'_'+str(i)+'.h5')
        print(model_path_info)
        y_pred = def_clustering_experiment(X_train[0], X_test[0], model_path_info, i_pgroups+1)
    
        
#         y_pred = model_trained.predict(X_test[0])
#         df_proba = pd.DataFrame(y_pred, columns=list(pd.DataFrame(ohe_kept.categories_).iloc[0,:]))
#         df_pred = pd.DataFrame(ohe_kept.inverse_transform(y_pred).reshape(1,-1)[0], columns=['prediction'])
        df_pred = pd.DataFrame(y_pred, columns=['prediction'])
        df_pred['ground_truth'] = y_test[0]
#         df_ground_truth = pd.DataFrame(ohe_kept.inverse_transform(np.array(y_test[0])).reshape(1,-1)[0], columns=['ground_truth'])
#         df_nn = pd.concat([df_proba, df_pred, df_ground_truth], axis=1)
        df_pred['experiment_index'] = i+1
        df_all_per_cell_out = pd.concat([df_all_per_cell_out , df_pred], axis=0)
    

    list_homo, list_comp, list_vmes, list_ari, list_ami, list_fm, list_acc = [],[],[],[],[],[],[]

    for i_exp in df_all_per_cell_out['experiment_index'].unique():
        df_temp = df_all_per_cell_out[(df_all_per_cell_out['experiment_index']==i_exp)]

        list_homo.append([ design_name, homogeneity_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'homogeneity'])
        list_comp.append([ design_name, completeness_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'completeness'])
        list_vmes.append([ design_name, v_measure_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'v_measure'])
        list_ari.append([ design_name, adjusted_rand_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'ari'])
        list_ami.append([ design_name, adjusted_mutual_info_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'ami'])
        list_fm.append([ design_name, fowlkes_mallows_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'fowlkes_mallows'])

    result = [element for lis in [list_homo, list_comp, list_vmes, list_ari, list_ami, list_fm] for element in lis]

    df_metric = pd.DataFrame(result, columns=['experiment','score','expr','metric'])


    df_result = pd.concat([df_result 
                           , pd.DataFrame(df_metric.groupby(['experiment', 'metric']).mean()['score']).pivot_table(index='experiment', columns='metric')])

df_result.to_csv(os.path.join(save_path, design+'.csv'))

4
EXPERIMENT --  1 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_0.h5
EXPERIMENT --  2 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_1.h5
EXPERIMENT --  3 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_2.h5
EXPERIMENT --  4 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_3.h5
EXPERIMENT --  5 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_4.h5
EXPERIMENT --  6 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_5.h5
EXPERIMENT --  7 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_6.h5
EXPERIMENT --  8 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_gene_space_1_layer_dense100+ppitf_7.h5
EXPERIMENT --  9 / 20
models/NN/exper_mouse/leavePgroupsOut/cell_out_4/full_ge

In [12]:
df_result

score            \
metric                                                   ami       ari   
experiment                                                               
full_gene_space_1_layer_dense100+ppitf-LeavePGr...  0.519579  0.435028   

                                                                 \
metric                                             completeness   
experiment                                                        
full_gene_space_1_layer_dense100+ppitf-LeavePGr...     0.719098   

                                                                    \
metric                                             fowlkes_mallows   
experiment                                                           
full_gene_space_1_layer_dense100+ppitf-LeavePGr...        0.702208   

                                                                          
metric                                             homogeneity v_measure  
experiment                                                                
full_gene_space_1_layer_dense100+ppitf-LeavePGr...    0.469917   0.53514